<a href="https://colab.research.google.com/github/Napawan2005/NLP-517432/blob/Test/LM_660710086.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**เหมือนกันตรงที่**
- ใช้แนวคิด transformer deep learning [1] เดียวกันทั้งหมด
- มีรูปแบบการตอบแบบ **text**[2]

**ข้อแตกต่าง**
- ตัวโมเดลใหญ่ขึ้น ข้อมูลถูกเทรนมากขึ้น

  - GPT-2 (2019) ข้อมูลถูกเทรน 1.5 พันล้าน

  - GPT-3(2020) ข้อมูลถูกเทรน 175 พันล้าน

  - GPT-4(2020) (ไม่เปิดเผย) คาดว่าระดับล้านล้าน


- มีจำนวนชั้น NN
  - GPT-2 (2019) = 48 Layers

  - GPT-3(2020) = 175 Layer

  - GPT-4(2020) (ไม่เปิดเผย) คาดว่า มากกว่า 175 layer

- Training Data (Amount and Sources)

  - GPT-2 (2019) = 40 GB of text from a variety of sources, including Wikipedia, web pages, and books
 - GPT-3(2020) = 570 GB of text from a variety of sources, including books, scientific papers, and web pages
 - GPT-4(2020) = Unknown, but likely even more diverse sources of text[2]



 อ้างอิง :
 - [1][www.ibm.com](https://www.ibm.com/think/topics/gpt)
 - [2][https://iq.opengenus.org/](https://iq.opengenus.org/gpt2-vs-gpt3-vs-gpt35-vs-gpt4/)




### ทดลองทำกับภาษาไทย

โดยใช้ชุดข้อมูลภาษาไทย เช่น PyThaiNLP สำหรับ Tokenization และนำมาสร้าง n-grams

In [2]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 38.4 MB/s eta 0:00:00


In [14]:
from datasets import load_dataset


In [18]:
dataset = load_dataset("prachathai67k")

README.md: 0.00B [00:00, ?B/s]

wisesight_sentiment/train-00000-of-00001(…):   0%|          | 0.00/2.58M [00:00<?, ?B/s]

wisesight_sentiment/validation-00000-of-(…):   0%|          | 0.00/286k [00:00<?, ?B/s]

wisesight_sentiment/test-00000-of-00001.(…):   0%|          | 0.00/327k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21628 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2404 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2671 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['texts', 'category'],
        num_rows: 21628
    })
    validation: Dataset({
        features: ['texts', 'category'],
        num_rows: 2404
    })
    test: Dataset({
        features: ['texts', 'category'],
        num_rows: 2671
    })
})


In [20]:
print("--- ตัวอย่างข้อมูลแรกจาก Dataset (ส่วน train) ---")
print(dataset['train'][0])
print("\n--- ฟีเจอร์ของ Dataset ---")
print(dataset['train'].features)

--- ตัวอย่างข้อมูลแรกจาก Dataset (ส่วน train) ---
{'texts': 'ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกรานต์เลย รอขับอยู่นาจา กระทัดรัด เหมาะกับสาวๆขับรถคนเดียวแบบเรา ราคาสบายกระเป๋า ประหยัดน้ำมัน วิ่งไกลแค่ไหนหายห่วงค่ะ', 'category': 1}

--- ฟีเจอร์ของ Dataset ---
{'texts': Value('string'), 'category': ClassLabel(names=['pos', 'neu', 'neg', 'q'])}


In [25]:
import re
import pandas as pd
from datasets import load_dataset
from pythainlp import word_tokenize
from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords

In [27]:
# 1. โหลด Dataset (ถ้ายังไม่ได้โหลด)
# แก้ไข: เปลี่ยนจาก 'prachathai67k' เป็น 'wisesight_sentiment' เพื่อหลีกเลี่ยง RuntimeError
dataset = load_dataset("wisesight_sentiment")

# แปลงเป็น Pandas DataFrame เพื่อให้จัดการง่าย (แนะนำสำหรับมือใหม่)
df = pd.DataFrame(dataset['train'])

# ลองดูหน้าตาข้อมูล 5 แถวแรก
print("Columns:", df.columns)
# แก้ไข: ปรับชื่อคอลัมน์จาก ['title', 'body_text'] เป็น ['texts', 'category'] ให้ตรงกับ wisesight_sentiment
print(df[['texts', 'category']].head())

# ==========================================
# ฟังก์ชันทำความสะอาดและตัดคำ
# ==========================================

# โหลดรายการคำฟุ่มเฟือย (Stopwords) เช่น "ก็", "การ", "ความ"
stopwords = set(thai_stopwords())

def process_thai_text(text):
    if not text:
        return []

    # 1. Normalize: จัดการสระลอย สระจม ให้เป็นมาตรฐานเดียวกัน
    text = normalize(text)

    # 2. Remove Special Chars: ลบตัวเลข, เครื่องหมายวรรคตอน, URL (ปรับแก้ตามโจทย์)
    text = re.sub(r'[\n\r\t]', ' ', text) # เปลี่ยนขึ้นบรรทัดใหม่เป็นช่องว่าง
    text = re.sub(r'http\S+', '', text)   # ลบ URL

    # 3. Tokenize: ตัดคำด้วย engine 'newmm' (ยอดนิยมสุด)
    words = word_tokenize(text, engine='newmm', keep_whitespace=False)

    # 4. Remove Stopwords: ลบคำที่ไม่สื่อความหมายและคำสั้นเกินไป
    clean_words = [w for w in words if w not in stopwords and len(w) > 1]

    return clean_words

# ==========================================
# เริ่มประมวลผล (อาจใช้เวลาสักพักเพราะข้อมูลเยอะ)
# ==========================================

print("กำลังตัดคำ... (อาจใช้เวลา 1-2 นาที)")

# เลือกใช้คอลัมน์ 'body_text' หรือ 'title' ตามต้องการ
# แก้ไข: ปรับชื่อคอลัมน์จาก 'body_text' เป็น 'texts' ให้ตรงกับ wisesight_sentiment
df['tokens'] = df['texts'].apply(process_thai_text)

# ดูผลลัพธ์
# แก้ไข: ปรับชื่อคอลัมน์จาก 'body_text' เป็น 'texts' ให้ตรงกับ wisesight_sentiment
print(df[['texts', 'tokens']].head())

Columns: Index(['texts', 'category'], dtype='object')
                                               texts  category
0  ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...         1
1  เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...         1
2                           บัตรสมาชิกลดได้อีกไหมคับ         1
3                                สนใจ new mazda2ครับ         1
4                                                 😍😍         0
กำลังตัดคำ... (อาจใช้เวลา 1-2 นาที)
                                               texts  \
0  ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...   
1  เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...   
2                           บัตรสมาชิกลดได้อีกไหมคับ   
3                                สนใจ new mazda2ครับ   
4                                                 😍😍   

                                              tokens  
0  [จอง, แล้, วนา, จา, Mitsubishi, Attrage, สงกรา...  
1  [ศักราช, นายกฯ, แถลงข่าว, การแข่งขัน, ศึก, ช้า...  
2                         [บัต